## This example is based on Pandas [video tutorial](https://www.youtube.com/watch?v=DkjCaAMBGWM) by [Rob Mulla](https://www.youtube.com/@robmulla)

In [ ]:
import pandas as pd
import sys
print(sys.version)
print('pandas version', pd.__version__)

## Reading from files

In [ ]:
df = pd.read_csv("./datasets/flights_2022.csv")
print(df.shape)
df.head()

## Writing to files

In [ ]:
# Original dataset is too large to upload to github
# https://www.kaggle.com/code/robikscube/flight-cancellation-dataset-eda/input
# I sampled 100,000 rows and save it as a new file
df2 = df.sample(1000)
print(df2.shape)
df2.head()
#df2.to_csv("./datasets/flights_2022.csv", index=False)

## Dataframe basics

In [ ]:
pd.set_option("display.max_columns", 500)

In [ ]:
df.head(3)

In [ ]:
df.tail(3)

In [ ]:
df.sample(3, random_state=42)

In [ ]:
df.sample(frac=0.1, random_state=42)

In [ ]:
df.columns

In [ ]:
df.index

## Dataframe summary

In [ ]:
df.info()

In [ ]:
df.info(verbose=False)

In [ ]:
df.describe()

In [ ]:
df[["Airline"]].describe()

In [ ]:
df.shape, len(df)

## Subsetting Columns

In [ ]:
df[['FlightDate', 'Airline', 'Origin']]

In [ ]:
df[df.columns[:5]]

In [ ]:
df[[c for c in df.columns if 'Time' in c]]

## Select dtypes

In [ ]:
df.select_dtypes('int')

## Select as series vs dataframe

In [ ]:
df['Airline']
print(type(df['Airline']))

In [ ]:
df[['Airline']]
print(type(df[['Airline']]))

## loc to subset rows and columns

In [ ]:
# Get the first 5 rows of the Airline and Origin columns
df.loc[:4, ['Airline', 'Origin']]

## loc filter expressions

In [ ]:
df.loc[df['Airline'] == 'Spirit Air Lines']

In [ ]:
df.loc[(df['Airline'] == 'Spirit Air Lines') & (df['FlightDate'] == '2022-03-30')]

In [ ]:
df.loc[~((df['Airline'] == 'Spirit Air Lines') & (df['FlightDate'] == '2022-03-30'))]

## query

In [ ]:
df.query('DepTime > 1130 and (Origin == "DRO")')

In [ ]:
# min_time = 1130
# df.query('DepTime > @min_time and (Origin == "DRO")')  # ValueError: Expression ((DepTime) > (__pd_eval_local_min_time)) & (Series_1_0x200x4c0x3a0x6c0xbf0x7d0x00x0) has forbidden control characters.

min_time = 1130
query_expression = 'DepTime > {} and (Origin == "DRO")'.format(min_time)
df.query(query_expression)

## Summary Statistics

In [ ]:
df['DepTime'].min()

In [ ]:
df['DepTime'].max()

In [ ]:
df['DepTime'].mean()

In [ ]:
df['DepTime'].std()

In [ ]:
df['DepTime'].sum()

In [ ]:
df['DepTime'].quantile(0.5)

In [ ]:
df['DepTime'].quantile([0.25, 0.75])

## agg

In [ ]:
df[['DepTime', 'DepDelay', 'ArrTime', 'ArrDelay']].agg(['mean', 'min', 'max'])

In [ ]:
df[['DepTime', 'DepDelay', 'ArrTime', 'ArrDelay']].agg(
    {
        'DepTime': ['min', 'max'],
        'DepDelay': ['mean'],
        'ArrTime': ['min', 'max']
    }
)

## Summarize categoricals

In [ ]:
df['Airline'].unique()

In [ ]:
distinct_count = df['Airline'].nunique()
print(distinct_count)

In [ ]:
df['Airline'].value_counts()

In [ ]:
df[['Airline', 'Origin']].value_counts()

In [ ]:
df[['Airline', 'Origin']].value_counts().reset_index()

## Clip

In [ ]:
# Limit the values to a specific range
print(df['DepTime'].min(), df['DepTime'].max())
df['DepTime'].clip(1000, 2000)

## Groupby

In [ ]:
df.groupby('Airline')[['DepDelay']].mean()

In [ ]:
df.groupby('Airline')[['DepDelay']].agg(['mean', 'min', 'max'])

In [ ]:
df_agg = df.groupby('Airline')[['DepDelay', 'ArrDelay']].agg(['mean', 'min', 'max'])
df_agg.head()

In [ ]:
df_agg.columns

In [ ]:
df_agg.columns = df_agg.columns.to_flat_index()
print(df_agg.columns)
df_agg.head()

In [ ]:
df_agg = df.groupby('Airline')[['DepDelay', 'ArrDelay']].agg(['mean', 'min', 'max'])
df_agg.columns = ["_".join(c) for c in df_agg.columns]
df_agg.columns

In [ ]:
df_agg

## New columns

In [ ]:
df['DepTime2'] = df['DepTime'] / 60
df.head(3)

## Sorting

In [ ]:
df.sort_values('ArrDelay')

In [ ]:
df[['FlightDate', 'Airline', 'ArrDelay']].sort_values('ArrDelay', ascending=False)

In [ ]:
df[['FlightDate', 'Airline', 'ArrDelay']].sort_values('ArrDelay', ascending=False).reset_index(drop=True)

In [ ]:
df_agg = df.groupby('Airline')[['DepDelay', 'ArrDelay']].agg(['mean', 'min', 'max'])
df_agg.sort_index(ascending=False)

## Missing data

In [ ]:
df[['FlightDate', 'Airline', 'ArrDelay']].isna()

In [ ]:
df[['FlightDate', 'Airline', 'ArrDelay']].isna().sum()

In [ ]:
df[['FlightDate', 'Airline', 'ArrDelay']].dropna(subset=['ArrDelay'])

In [ ]:
df[['FlightDate', 'Airline', 'ArrDelay']].fillna(-999)

In [ ]:
df['ArrDelay'].fillna(df['ArrDelay'].mean())

## Concat

In [ ]:
df1 = df[['FlightDate', 'Airline', 'Origin', 'Dest']].loc[df['Airline'] == "Southwest Airlines Co."].copy()
df2 = df[['FlightDate', 'Airline', 'Origin', 'Dest']].loc[df['Airline'] == "Delta Air Lines Inc."].copy()

In [ ]:
pd.concat([df1, df2])

In [ ]:
df_stack = pd.concat([df1, df2])
df_stack

In [ ]:
df1 = df1.reset_index(drop=True)
df2 = df2.reset_index(drop=True)
df1.head()

In [ ]:
df_side = pd.concat([df1, df2], axis=1)
df_side.head()

In [ ]:
df1.shape, df2.shape, df_stack.shape, df_side.shape